In [4]:


%load_ext autoreload
%autoreload 2

from collections import defaultdict, Counter
import numpy as np
import itertools
import copy
import matplotlib.pyplot as plt
import json
import time
from dataraw_sampling_SETShatter import sample_queries, sample_queries_simple_shatter, resolve_eval_expression, decode_key_to_vocab_token



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
num_attributes=5; num_attr_vals=3; query_length_multiplier=1; nest_depth_int=1; multiple_OR_sets_bool=True
N_train=0; N_val=5120; N_test=5120

expression_resolve_fn = resolve_eval_expression

sample_start_time = time.time()
data, stats = sample_queries_simple_shatter(
    num_attributes, query_length_multiplier, nest_depth_int, multiple_OR_sets_bool, 
    expression_resolve_fn,
    N_train, N_val, N_test,
    validate=True,
    debug=False)
sample_end_time = time.time()

print(f'Sample Time for {N_train+N_val+N_test} datapoints = {sample_end_time-sample_start_time} seconds')


save_start_time = time.time()
filename1 = '../Raw_Datasets/SET/SimpleShatter-{}Attr-{}Vals-{}LenMultiplier-{}NestDepth-{}Train-{}Val-{}Test.json'.format(
    num_attributes, num_attr_vals, query_length_multiplier, nest_depth_int, N_train, N_val, N_test)
with open(filename1, 'w') as f:
    json.dump(data, f)
save_end_time = time.time()

filename = '../Raw_Datasets/SET/SimpleShatter-{}Attr-{}Vals-{}LenMultiplier-{}NestDepth-{}Train-{}Val-{}Test-Stats.json'.format(
    num_attributes, num_attr_vals, query_length_multiplier, nest_depth_int, N_train, N_val, N_test)
with open(filename, 'w') as f:
    json.dump(stats, f)
save_end_time = time.time()

print(f'Saving Time for {N_train+N_val+N_test} datapoints = {save_end_time-save_start_time} seconds')

100%|██████████| 10240/10240 [00:00<00:00, 18241.04it/s]


Sample Time for 10240 datapoints = 0.5687417984008789 seconds
Saving Time for 10240 datapoints = 1.2271051406860352 seconds


In [1]:
3**5

134217728

In [6]:
filename1

'../Raw_Datasets/SET/SimpleShatter-5Attr-3Vals-1LenMultiplier-1NestDepth-0Train-5120Val-5120Test.json'

In [40]:
data.keys()

dict_keys(['num_attributes', 'num_attr_vals', 'nest_depth_int', 'key_support_size', 'multiple_OR_sets_bool', 'query_length_multiplier', 'max_len_q', 'len_k', 'train_gt_idxs', 'val_gt_idxs', 'test_gt_idxs', 'train_tokens', 'val_tokens', 'test_tokens', 'vocab_size', '(', ')', 'NULL', 'SEP', 'SOS', 'EOS', 'PAD', 'PLH', '&', '|'])

In [6]:
data['val_tokens']

[([0], [0]),
 ([1], [1]),
 ([2], [2]),
 ([3], [3]),
 ([4], [4]),
 ([5], [5]),
 ([6], [6]),
 ([7], [7]),
 ([8], [8]),
 ([0, 1], [0]),
 ([0, 1], [1]),
 ([0, 2], [0]),
 ([0, 2], [2]),
 ([0, 3], [0]),
 ([0, 3], [3]),
 ([0, 4], [0]),
 ([0, 4], [4]),
 ([0, 5], [0]),
 ([0, 5], [5]),
 ([0, 6], [0]),
 ([0, 6], [6]),
 ([0, 7], [0]),
 ([0, 7], [7]),
 ([0, 8], [0]),
 ([0, 8], [8]),
 ([1, 2], [1]),
 ([1, 2], [2]),
 ([1, 3], [1]),
 ([1, 3], [3]),
 ([1, 4], [1]),
 ([1, 4], [4]),
 ([1, 5], [1]),
 ([1, 5], [5]),
 ([1, 6], [1]),
 ([1, 6], [6]),
 ([1, 7], [1]),
 ([1, 7], [7]),
 ([1, 8], [1]),
 ([1, 8], [8]),
 ([2, 3], [2]),
 ([2, 3], [3]),
 ([2, 4], [2]),
 ([2, 4], [4]),
 ([2, 5], [2]),
 ([2, 5], [5]),
 ([2, 6], [2]),
 ([2, 6], [6]),
 ([2, 7], [2]),
 ([2, 7], [7]),
 ([2, 8], [2]),
 ([2, 8], [8]),
 ([3, 4], [3]),
 ([3, 4], [4]),
 ([3, 5], [3]),
 ([3, 5], [5]),
 ([3, 6], [3]),
 ([3, 6], [6]),
 ([3, 7], [3]),
 ([3, 7], [7]),
 ([3, 8], [3]),
 ([3, 8], [8]),
 ([4, 5], [4]),
 ([4, 5], [5]),
 ([4, 6], [4]),
 ([

In [5]:
data['val_gt_idxs']

[[1, 2, 6, 3, 8],
 [3, 7, 5, 2],
 [5, 4, 1, 8, 2, 6],
 [0, 2, 5, 3],
 [6, 7, 3, 0, 1, 4],
 [0, 3, 2, 4, 6, 1],
 [2, 4, 5, 0],
 [4, 6, 3, 8],
 [4, 8, 6, 1],
 [8, 5, 0],
 [3, 2, 7],
 [3, 0, 8, 7, 1, 6],
 [1],
 [8, 6, 4, 5, 0, 3],
 [1, 7, 2, 3],
 [7, 0, 3, 5, 4],
 [8, 3, 4, 6, 5],
 [3, 2, 7],
 [5, 3, 0],
 [8, 5],
 [6, 2, 7, 5, 4, 8, 0],
 [5, 7, 6, 8],
 [1, 5, 8, 3, 7],
 [3, 2, 8, 1, 5, 0],
 [4, 7],
 [1, 0, 3, 4, 2],
 [0, 2, 7],
 [8, 3, 4, 6],
 [5, 7, 1, 2],
 [1, 0, 3, 4, 8],
 [8],
 [5, 0, 3, 4],
 [7, 2, 1, 6],
 [5, 6, 0, 8, 7, 4],
 [6, 1, 7, 8],
 [3, 4, 2, 5, 7],
 [5, 6, 8],
 [2, 0, 7, 3, 8],
 [3, 0, 4, 8],
 [7, 1, 3],
 [8, 7, 0, 1, 5, 2],
 [1, 0, 3, 2, 7, 6],
 [3, 4, 1, 7, 2, 0, 8, 5, 6],
 [4, 8, 6, 1, 5],
 [0, 5, 8, 1, 6, 3],
 [2, 4, 6],
 [4],
 [8, 7, 1, 6, 4, 3],
 [6, 8, 4, 5, 7, 0, 2],
 [7, 1, 8, 0, 4],
 [1, 2, 5],
 [2, 0, 8, 4, 7],
 [3, 7, 2],
 [4, 6, 7, 8, 0, 3, 2],
 [7, 5, 3, 6],
 [1, 8, 3],
 [1, 0, 6, 5],
 [0, 1],
 [5, 6, 2, 8, 4],
 [0, 6, 4, 1, 5, 7, 3, 2, 8],
 [8, 5, 4, 2, 7, 1,

In [118]:
data['val_tokens']


[([0], [0]),
 ([1], [1]),
 ([2], [2]),
 ([3], [3]),
 ([4], [4]),
 ([5], [5]),
 ([6], [6]),
 ([7], [7]),
 ([8], [8]),
 ([0, 1], [0]),
 ([0, 1], [1]),
 ([0, 2], [0]),
 ([0, 2], [2]),
 ([0, 3], [0]),
 ([0, 3], [3]),
 ([0, 4], [0]),
 ([0, 4], [4]),
 ([0, 5], [0]),
 ([0, 5], [5]),
 ([0, 6], [0]),
 ([0, 6], [6]),
 ([0, 7], [0]),
 ([0, 7], [7]),
 ([0, 8], [0]),
 ([0, 8], [8]),
 ([1, 2], [1]),
 ([1, 2], [2]),
 ([1, 3], [1]),
 ([1, 3], [3]),
 ([1, 4], [1]),
 ([1, 4], [4]),
 ([1, 5], [1]),
 ([1, 5], [5]),
 ([1, 6], [1]),
 ([1, 6], [6]),
 ([1, 7], [1]),
 ([1, 7], [7]),
 ([1, 8], [1]),
 ([1, 8], [8]),
 ([2, 3], [2]),
 ([2, 3], [3]),
 ([2, 4], [2]),
 ([2, 4], [4]),
 ([2, 5], [2]),
 ([2, 5], [5]),
 ([2, 6], [2]),
 ([2, 6], [6]),
 ([2, 7], [2]),
 ([2, 7], [7]),
 ([2, 8], [2]),
 ([2, 8], [8]),
 ([3, 4], [3]),
 ([3, 4], [4]),
 ([3, 5], [3]),
 ([3, 5], [5]),
 ([3, 6], [3]),
 ([3, 6], [6]),
 ([3, 7], [3]),
 ([3, 7], [7]),
 ([3, 8], [3]),
 ([3, 8], [8]),
 ([4, 5], [4]),
 ([4, 5], [5]),
 ([4, 6], [4]),
 ([

In [10]:
ref_p = '/app/Contrastive-Learning-Benchmarking/Raw_Datasets/SimpleShatter/SimpleShatter-Full-1Train-10Val.json'

with open(ref_p, 'r') as f:
    ref = json.load(f)

In [11]:
ref.keys()

dict_keys(['num_attributes', 'num_attr_vals', 'num_cards_per_query', 'nest_depth_int', 'key_support_size', 'query_support_size', 'max_len_q', 'len_k', 'hold_out', 'train_datapoints', 'val_datapoints', 'train_tokens', 'val_tokens', 'sparsity_estimate', 'vocab_size', '(', ')', 'NULL', 'SEP', 'SOS', 'EOS', 'PAD', 'PLH'])

In [39]:
ref['train_tokens'][-200:-100]

[[[0, 2, 3, 4, 6, 7, 8, 9], [6]],
 [[0, 2, 3, 4, 6, 7, 8, 9], [7]],
 [[0, 2, 3, 4, 6, 7, 8, 9], [8]],
 [[0, 2, 3, 4, 6, 7, 8, 9], [9]],
 [[0, 2, 3, 5, 6, 7, 8, 9], [0]],
 [[0, 2, 3, 5, 6, 7, 8, 9], [2]],
 [[0, 2, 3, 5, 6, 7, 8, 9], [3]],
 [[0, 2, 3, 5, 6, 7, 8, 9], [5]],
 [[0, 2, 3, 5, 6, 7, 8, 9], [6]],
 [[0, 2, 3, 5, 6, 7, 8, 9], [7]],
 [[0, 2, 3, 5, 6, 7, 8, 9], [8]],
 [[0, 2, 3, 5, 6, 7, 8, 9], [9]],
 [[0, 2, 4, 5, 6, 7, 8, 9], [0]],
 [[0, 2, 4, 5, 6, 7, 8, 9], [2]],
 [[0, 2, 4, 5, 6, 7, 8, 9], [4]],
 [[0, 2, 4, 5, 6, 7, 8, 9], [5]],
 [[0, 2, 4, 5, 6, 7, 8, 9], [6]],
 [[0, 2, 4, 5, 6, 7, 8, 9], [7]],
 [[0, 2, 4, 5, 6, 7, 8, 9], [8]],
 [[0, 2, 4, 5, 6, 7, 8, 9], [9]],
 [[0, 3, 4, 5, 6, 7, 8, 9], [0]],
 [[0, 3, 4, 5, 6, 7, 8, 9], [3]],
 [[0, 3, 4, 5, 6, 7, 8, 9], [4]],
 [[0, 3, 4, 5, 6, 7, 8, 9], [5]],
 [[0, 3, 4, 5, 6, 7, 8, 9], [6]],
 [[0, 3, 4, 5, 6, 7, 8, 9], [7]],
 [[0, 3, 4, 5, 6, 7, 8, 9], [8]],
 [[0, 3, 4, 5, 6, 7, 8, 9], [9]],
 [[1, 2, 3, 4, 5, 6, 7, 8], [1]],
 [[1, 2, 3, 4,

In [26]:
import random

l = [2, 3, 4, 5, 6, 7, 8, 9]
random.shuffle(l)

In [27]:
l


[2, 8, 9, 7, 5, 3, 6, 4]

In [107]:
from collections import defaultdict, Counter
import itertools
import copy
import random
import json
import operator as op
from functools import reduce
from tqdm import tqdm
from itertools import chain, combinations
import numpy as np
import time

In [108]:
num_keys = 9
s = [i for i in range(num_keys)]
power_set = list(chain.from_iterable(combinations(s, r) for r in range(len(s)+1)))
all_subsets = power_set[1:]
probs = np.array([len(subset) for subset in all_subsets]) / np.sum(np.array([len(subset) for subset in all_subsets]))

In [69]:
tokens = []
gt_idxs = []
buckets = {}
or_ops = {}
and_ops = {}
depths = {}
input_lens = {}

max_len_q = 0
for i in tqdm(range(2304)):
    gt_ks_idx = list(np.random.choice(a=all_subsets, p=probs))
    bucket = len(gt_ks_idx)
    k_idx = int(np.random.choice(gt_ks_idx))
    
    q_tokens = gt_ks_idx
    num_or_ops = 0
    num_and_ops = 0
    depth = 0
            
    # accumulate datapoints
    tokens.append((q_tokens, [k_idx]))
    gt_idxs.append(gt_ks_idx)

    # stats
    max_len_q = max(max_len_q, len(q_tokens))
    input_lens[len(q_tokens)] = input_lens.get(len(q_tokens), 0) + 1
    buckets[bucket] = buckets.get(bucket, 0) + 1
    or_ops[num_or_ops] = or_ops.get(num_or_ops, 0) + 1
    and_ops[num_and_ops] = and_ops.get(num_and_ops, 0) + 1
    depths[depth] = depths.get(depth, 0) + 1    

  0%|          | 0/2304 [00:00<?, ?it/s]<ipython-input-69-6e5eb9fa616d>:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  gt_ks_idx = list(np.random.choice(a=all_subsets, p=probs))
100%|██████████| 2304/2304 [00:00<00:00, 5174.55it/s]


In [71]:
buckets

{7: 231, 6: 504, 5: 633, 2: 81, 3: 268, 4: 504, 9: 15, 8: 64, 1: 4}

In [74]:
tokens = []
gt_idxs = []
buckets = {}
or_ops = {}
and_ops = {}
depths = {}
input_lens = {}

max_len_q = 0
for i in tqdm(range(len(all_subsets))):

    # each query column hit a number of keys
    # gt_ks_idx, bucket = sample_keys_column(num_attributes, num_attr_vals, bucket_probs)
    # gt_ks_idx = list(np.random.choice(a=all_subsets, p=probs))
    gt_ks_idx = list(all_subsets[i])
    bucket = len(gt_ks_idx)
    for k_idx in gt_ks_idx:
        # # draw one key from this column of keys
        # k_idx = int(np.random.choice(gt_ks_idx))

        q_tokens = gt_ks_idx
        num_or_ops = 0
        num_and_ops = 0
        depth = 0

        # accumulate datapoints
        tokens.append((q_tokens, [k_idx]))
        gt_idxs.append(gt_ks_idx)

        # stats
        max_len_q = max(max_len_q, len(q_tokens))
        input_lens[len(q_tokens)] = input_lens.get(len(q_tokens), 0) + 1
        buckets[bucket] = buckets.get(bucket, 0) + 1
        or_ops[num_or_ops] = or_ops.get(num_or_ops, 0) + 1
        and_ops[num_and_ops] = and_ops.get(num_and_ops, 0) + 1
        depths[depth] = depths.get(depth, 0) + 1 

100%|██████████| 511/511 [00:00<00:00, 143922.20it/s]


In [75]:
buckets

{1: 9, 2: 72, 3: 252, 4: 504, 5: 630, 6: 504, 7: 252, 8: 72, 9: 9}

In [82]:
stats['bucket_counts']

{6: 524, 7: 252, 3: 229, 4: 527, 5: 608, 2: 73, 8: 71, 9: 13, 1: 7}